This creates a long file with clip FT magnitudes transposed and concatenated into a long series of samples

It's designed for a process that just grabs a window instead of using a prepared frame (since that creates massive data duplication)

In [1]:
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
import time
import cmath

In [17]:
# Get shared constants and functions
%run "NN Audio Core.py"

In [18]:
# Number of all zero samples between clips
PAD_SIZE = 50

In [19]:
def frames_file(data_path, max_samples):
    frames_file = data_path + "\\fsamples-MP" + str(SAMPLE_BINS) + "-" + str(max_samples)
    filename = os.fsdecode(frames_file)
    return filename

def targets_file(data_path, max_samples):
    targets_file = data_path + "\\ftargets-C" + str(SAMPLE_BINS) + "-" + str(max_samples)
    filename = os.fsdecode(targets_file)
    return filename

# Iterate over clean & noisy folders to create frames and targets
# Updated to select clips based on difference between clean & noisy versions
def create_data(wav_root, data_path, max_samples = 10000, min_diff = 70, max_diff=300, mmap=True):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    sample_index = 0
    if (mmap):
        frames = np.memmap(frames_file(data_path, max_samples), mode='w+', dtype=DTYPE, shape=(max_samples,SAMPLE_BINS,INPUT_DEPTH))
        targets = np.memmap(targets_file(data_path, max_samples), mode='w+', dtype=OUTPUT_DTYPE, shape=(max_samples,SAMPLE_BINS,OUTPUT_DEPTH))
    else:
        frames = np.empty(dtype=DTYPE, shape=(max_samples,SAMPLE_BINS,INPUT_DEPTH))
        targets = np.empty(dtype=OUTPUT_DTYPE, shape=(max_samples,SAMPLE_BINS,OUTPUT_DEPTH))
    file_list = os.listdir(clean_dir)
    file_index = 0
    while (sample_index < max_samples) and (file_index < len(file_list)) :
        file = file_list[file_index]
        filename = os.fsdecode(file)
        file_index += 1
        print("file ", noisy_dir+file)
        noisy_ft = get_ft_from_file(noisy_dir + file)
        clean_ft = get_ft_from_file(clean_dir + file)
        diff = diff_ft(noisy_ft, clean_ft)
        print("diff ", diff)
        if (diff > min_diff) and (diff < max_diff):
            new_frames = get_samples(noisy_dir + file)
            max_step = min(new_frames.shape[0], max_samples-sample_index)
            frames[sample_index:sample_index+max_step,:,:] = new_frames[:max_step,:,:]
            new_targets = get_targets(clean_dir + file)
            targets[sample_index:sample_index+max_step,:,:] = new_targets[:max_step,:,:]
            sample_index += new_targets.shape[0]
            if (PAD_SIZE > 0) and (sample_index < max_samples):
                pad_width = min(PAD_SIZE, max_samples-sample_index)
                frames_pad = np.zeros((pad_width, SAMPLE_BINS,INPUT_DEPTH))
                targets_pad = np.zeros((pad_width, SAMPLE_BINS,OUTPUT_DEPTH))
                frames[sample_index:sample_index+pad_width,:,:] = frames_pad[:pad_width,:]
                targets[sample_index:sample_index+pad_width,:,:] = targets_pad[:pad_width,:]
                sample_index += pad_width
            print("si ", sample_index)
    print("Reached sample # " + str(min(sample_index, max_samples)))
    return frames, targets

In [20]:
# small test data "Assets\\DataShareArchive\\Test"
# 28K "F:\\Audiodata\\Train28Spk"
f, t = create_data("F:\\Audiodata\\Train28Spk", "f:\\Audiodata", max_samples=100, min_diff=0.05, max_diff=0.25, mmap=False)

file  F:\Audiodata\Train28Spk\Noisy\p226_001.wav
diff  0.16292214


IndexError: too many indices for array

In [ ]:
# To release the files if we need to
del f, t